# `Step 4: Outlier`
- Free
- Z Score
- ISO
- LOF 

# 0- `Config`

In [ ]:
import pandas as pd 
import numpy as np 
import pickle 

In [ ]:
try: 
    if manager == 1:
        print("s1_load.ipynb running from MANAGER")
except: 
    %run s0_config.ipynb 
    subject_outlier_0 = "ZScore_v0"  
    subject_outlier_1 = "LOF_v0" 

# 1- `Load`

In [ ]:
import_name = "data_raw"
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_load}", f"{import_name}" + ".pkl")) , 'rb') as file:
    data_raw = pickle.load(file) 

import_name = "data_full"
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_load}", f"{import_name}" + ".pkl")) , 'rb') as file:
    data_full = pickle.load(file) 

---
# 2- `Preprocessing`

### 0-) `Overview`

In [ ]:
print(f"Subject Outlier of Embeddings: {subject_outlier_0}")
print(f"Subject Outlier of Expression: {subject_outlier_1}") 
print(f"Shape: {data_full.shape}") 
print(f"Number of unique cell types: {len(pd.unique(data_full['Cell_Type']))}")  
print(f"Unique cell types: {pd.unique(data_full['Cell_Type'])}")  
pd.concat([data_full.head(2), data_full.tail(2)], axis=0) 

### 1-) `Tool Setup`

In [ ]:
outlier_tool = access_data_path(f"{process_barcode}/class/{folder_outlier}", "tool" + ".ipynb") 
%run $outlier_tool 

### 2-) `Convert`

In [ ]:
# We need to convert the data to numpy arrays
Embeddings       = np.stack(data_full['Embeddings'].values) 
# However, as there will be subject_data_full after the first process, this expression sample is just for demonstration. Besides it needs to be reshaped in future. 
#Expression_Matrix                             = data_full['Expression'].values 
# BERT does not have second matrix 
#label_embeddings_matrix        = np.stack(data_full['Cell_Type_Embeddings'].values)   

In [ ]:
data_full

---
# 3- `Bench`

### 1-) `Embeddings`

#### 0-) `Oveview`

In [ ]:
data_full

#### 1-) `Operation`

In [ ]:
# Temporarly for operation 
target_embeddings_matrix = Embeddings.copy() 
subject_outlier          = subject_outlier_0

# Input data: data_full
data_embedded = data_full.copy() 
outlier_operation = access_data_path(f"{process_barcode}/class/{folder_outlier}", "operation" + ".ipynb") 
%run $outlier_operation 
data_full = data_embedded.copy() 
# Output: subject_data_full 

#### 3-) `Report`

In [ ]:
print(f"Shape (Before): {data_full.shape}") 
print(f"Shape (After): {subject_data_full.shape}")  
print(f"Discarded rows: {data_full.shape[0] - subject_data_full.shape[0]}")  

print(f"Number of unique cell types: {len(pd.unique(data_full['Cell_Type']))}")  
print(f"Number of unique cell types: {len(pd.unique(subject_data_full['Cell_Type']))}")  
print(f"Lost cell types: {set(pd.unique(data_full['Cell_Type'])) - set(pd.unique(subject_data_full['Cell_Type']))}") 


### 2-) `Cell_Type_Gene_Marker_Embeddings`

#### 0-) `Oveview`

In [ ]:
data_full

#### 1-) `Operation`

In [ ]:
Expression_Matrix          = subject_data_full['Expression'].values 

Expression_Matrix_Reshaped = Expression_Matrix.reshape(-1, 1) # Necessary for the pipeline  

In [ ]:
# Temporarly for operation 
target_embeddings_matrix = Expression_Matrix_Reshaped.copy() 
subject_outlier          = subject_outlier_1

# Input data: data_full
data_embedded = subject_data_full.copy() 
outlier_operation = access_data_path(f"{process_barcode}/class/{folder_outlier}", "operation" + ".ipynb") 
%run $outlier_operation 
data_full = data_embedded.copy() 
# Output: subject_data_full 

#### 2-) `Report`

In [ ]:
print(f"Shape (Before): {data_embedded.shape}") 
print(f"Shape (After): {subject_data_full.shape}")  
print(f"Discarded rows: {data_embedded.shape[0] - subject_data_full.shape[0]}")  

print(f"Number of unique cell types: {len(pd.unique(data_embedded['Cell_Type']))}")  
print(f"Number of unique cell types: {len(pd.unique(subject_data_full['Cell_Type']))}")  
print(f"Lost cell types: {set(pd.unique(data_embedded['Cell_Type'])) - set(pd.unique(subject_data_full['Cell_Type']))}") 


In [ ]:
subject_data_full

---
# 4- `End`

In [ ]:
print(f"Subject Outlier 0 (Embedding): {subject_outlier_0}")
print(f"Subject Outlier 1 (Expression): {subject_outlier_1}")

In [ ]:
export_name = "data_raw" 
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_outlier}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(data_raw, file)   

export_name = "subject_data_full" 
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_outlier}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(subject_data_full, file)   
    
export_name = "subject_outlier_0" 
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_outlier}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(subject_outlier_0, file)   

export_name = "subject_outlier_1" 
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_outlier}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(subject_outlier_1, file)   